# Libraries
<br><br>
Importing all the required libraries

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch
import time
from scipy.spatial.distance import cdist
from numpy.linalg import svd
import random

# Check GPU

<br>

## Write a little code to see the difference between CPU and GPU

My CPU: Intel® Core™ i7-10875H CPU @ 2.30GHz <br>
My GPU: NVIDIA GeForce RTX 2080 Super

In [ ]:
# Determine device to run on ( GPU vs CPU )
device = torch.device( "cuda" if torch.cuda.is_available() else "cpu")
print ("Running tensors in ", device)

matrix_size = 32*32
x = torch.rand(matrix_size, matrix_size)
y = torch.rand(matrix_size, matrix_size)

print("******************* CPU *******************")
start = time.time()
result = torch.matmul(x,y)
print(time.time() - start)
print("Verify device: ", result.device)

x_gpu = x.to(device)
y_gpu = y.to(device)
torch.cuda.synchronize()

for i in range(3):
    print("******************* GPU *******************")
    start = time.time()
    result_gpu = torch.matmul(x_gpu,y_gpu)
    torch.cuda.synchronize()
    print(time.time() - start)
    print("Verify device: ", result_gpu.device)


# Useful Functions

In [ ]:
# numpy array to torch tensor
def to_torch_tensor(x, device="cuda", dtype=torch.float32, requires_grad=False):
    # return torch . from_numpy ( x ) . to ( device )
    return torch.tensor(x, dtype=dtype, requires_grad=requires_grad).to(device)

# torch tensor to numpy


def to_numpy_array(x):
    return x.detach().cpu().numpy()


# Part I - Extracting Features and Matching using OpenCV

In [ ]:

# def extract_features_gpu(image):
#     # Create a SIFT object and set the device to use the GPU
#     sift = cv2.cuda_DescriptorMatcher.createBFMatcher(cv2.NORM_L2)
#     sift.setPurgeScaleFactor(10)
#     sift.setUseScaleOrientation(True)

#     # Convert the input image to a GPU Mat object
#     image_gpu = cv2.cuda_GpuMat()
#     image_gpu.upload(image)

#     # Detect keypoints and compute descriptors using SIFT on the GPU
#     keypoints_gpu, descriptors_gpu = sift.detectWithDescriptors(image_gpu, None)

#     # Download the keypoints and descriptors from the GPU to the CPU
#     keypoints = keypoints_gpu.download()
#     descriptors = descriptors_gpu.download()

#     return keypoints, descriptors


## Extracting keypoints and features using SIFT

In [ ]:
def extract_features(image):
    """
    Extracts SIFT features from an image.

    Args:
        image: input image

    Returns:
        keypoints: list of SIFT keypoints
        descriptors: SIFT descriptors for the keypoints
    """
    sift = cv2.xfeatures2d.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)

    return keypoints, descriptors


## Extracting the best 1000 keypoints manually

In [ ]:
def extract_best_N_features_manually(keypoints, descriptors, n):
    """
    Extracts the top N SIFT features from an image based on their response.

    Args:
        keypoints: list of SIFT keypoints
        descriptors: SIFT descriptors for the keypoints
        n: number of top features to select

    Returns:
        keypoints_np: Numpy array of keypoints
        N_keypoints: list of top N SIFT keypoints
        N_features: SIFT descriptors for the top N keypoints
    """
    # Sort keypoints by their response and select the top N keypoints
    sorted_keypoints = sorted(
        zip(keypoints, descriptors), key=lambda x: x[0].response, reverse=True)
    N_keypoints, N_features = zip(*sorted_keypoints[:n])

    # Convert the keypoints to a Numpy array
    keypoints_np = cv2.KeyPoint_convert(keypoints)

    return keypoints_np, N_keypoints, N_features


## Extracting the best 1000 keypoints by changing the parameters of the SIFT.CREATE class
<br>

After reading the documentation of OpenCV's [**SIFT's**](https://docs.opencv.org/4.x/da/df5/tutorial_py_sift_intro.html) [**cv::SIFT Class**](docs.opencv.org/3.4/d7/d60/classcv_1_1SIFT.html), I found out all the parameters which we can change for SIFT. <br><br>

Moreover, I also found that we can do the same thing using **BRISK's** [**cv::BRISK Class**](https://docs.opencv.org/3.4/de/dbf/classcv_1_1BRISK.html)<br>

BRISK is a feature point detection and description algorithm with scale invariance and rotation invariance, developed in 2011 as a ***free alternative*** to **SIFT** and readily implemented in famous CV libraries such as OpenCV.

Both the implementations are as following:

In [ ]:
def extract_features_SIFT_parameters(image):
    """
    Extracts SIFT features from an image with custom parameters.

    Args:
        image: input image

    Returns:
        keypoints: list of SIFT keypoints
        descriptors: SIFT descriptors for the keypoints
    """
    sift = cv2.xfeatures2d.SIFT_create(contrastThreshold=0.02, nfeatures=1000)
    keypoints, descriptors = sift.detectAndCompute(image, None)

    return keypoints, descriptors


In [ ]:
def extract_features_BRISK_parameters(image):
    """
    Extracts BRISK features from an image with custom parameters.

    Args:
        image: input image

    Returns:
        keypoints: list of BRISK keypoints
        descriptors: BRISK descriptors for the keypoints
    """
    brisk = cv2.BRISK_create(thresh=30, octaves=3, patternScale=1.0)
    keypoints = brisk.detect(image, None)
    keypoints = sorted(keypoints, key=lambda x: -x.response)[:1000]
    keypoints, descriptors = brisk.compute(image, keypoints)

    return keypoints, descriptors


In [ ]:
def run_all_extractors(image):
    """
    Runs multiple feature extraction algorithms on an input image.

    Args:
        image: input image

    Returns:
        keypoints: list of all keypoints extracted
        descriptors: descriptors for all keypoints extracted
        keypoints_np: numpy array of the best N keypoints
        N_keypoints: list of the best N keypoints
        N_features: descriptors for the best N keypoints
        keypoints_SIFT: list of keypoints extracted using SIFT with custom parameters
        descriptors_SIFT: descriptors for keypoints extracted using SIFT with custom parameters
        keypoints_BRISK: list of keypoints extracted using BRISK with custom parameters
        descriptors_BRISK: descriptors for keypoints extracted using BRISK with custom parameters
    """
    # Pass the image to the SIFT feature extractor
    keypoints, descriptors = extract_features(image)
    print("Number of keypoints extracted in total:", len(keypoints))

    # Pass the keypoints to extract the 'N' best keypoints
    keypoints_np, N_keypoints, N_features = extract_best_N_features_manually(
        keypoints, descriptors, n=1000)
    print("Number of keypoints after manually extracting the best 1000 keypoints:", len(
        N_keypoints))

    # Pass the image to the SIFT feature extractor with the changed parameters
    keypoints_SIFT, descriptors_SIFT = extract_features_SIFT_parameters(image)
    print("Number of keypoints after changing the parameters of the SIFT function:", len(
        keypoints_SIFT))

    # Pass the image to the BRISK feature extractor with the changed parameters
    keypoints_BRISK, descriptors_BRISK = extract_features_BRISK_parameters(
        image)
    print("Number of keypoints using the BRISK class:", len(keypoints_BRISK))

    return keypoints, descriptors, keypoints_np, N_keypoints, N_features, keypoints_SIFT, descriptors_SIFT, keypoints_BRISK, descriptors_BRISK


## Match Finding using Brute Force (one to all)

In [ ]:
def find_matches(img1, img2, kp1, kp2, des1, des2, threshold):
    """
    Finds matching features between two images based on their keypoints and descriptors.

    Args:
        img1: first image
        kp1: keypoints of the first image
        des1: descriptors of the first image
        img2: second image
        kp2: keypoints of the second image
        des2: descriptors of the second image

    Returns:
        matches: list of matching features
        matched_image: image showing the matches between the two input images
    """
    # Check that the keypoints and descriptors are not empty
    if not (len(kp1) and len(kp2) and len(des1) and len(des2)):
        print("Empty keypoints or descriptors")
        return [], None

    # Compute the distances between each pair of descriptors
    distances = cdist(des1, des2, 'euclidean')

    # Set the threshold dynamically based on the distribution of distances
    threshold = np.median(distances) * threshold

    matches = []
    for i in range(len(des1)):
        # Find the index of the closest descriptor in the second image
        idx = np.argmin(distances[i])

        # If the distance is below the threshold, add the match
        distance = distances[i][idx]
        if distance < threshold:
            matches.append(cv2.DMatch(i, idx, distance))

    # Draw the matches as lines between corresponding key points
    if matches:
        matched_image = cv2.drawMatches(img1, kp1, img2, kp2, matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    else:
        matched_image = None

    return matches, matched_image

In [ ]:
def find_matches_1(img1, kp1, des1, img2, kp2, des2, threshold):
    """
    Finds matches between keypoints in two images using Euclidean distance.

    Args:
        img1: first image
        kp1: keypoints in first image
        des1: descriptors for keypoints in first image
        img2: second image
        kp2: keypoints in second image
        des2: descriptors for keypoints in second image

    Returns:
        matches: list of matched keypoints (cv2.DMatch objects)
        matched_image: image with matched keypoints drawn
    """
    # Check that the keypoints and descriptors are not empty
    if not (len(kp1) and len(kp2) and len(des1) and len(des2)):
        print("Empty keypoints or descriptors")
        return [], None

    # Compute the distances between each pair of descriptors
    distances = cdist(des1, des2, 'euclidean')

    # Set the threshold dynamically based on the distribution of distances
    threshold = np.median(distances) * threshold

    matches = []
    for i in range(len(kp1)):
        # Compute the distances between the descriptor i from image 1 and all descriptors from image 2
        distances_i = np.linalg.norm(des2 - des1[i], axis=1)

        if np.min(distances_i) < threshold:
            j = np.argmin(distances_i)
            matches.append(cv2.DMatch(i, j, np.min(distances_i)))

    matched_image = cv2.drawMatches(
        img1, kp1, img2, kp2, matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

    return matches, matched_image


Both functions *find_matches()* and *find_matches_1()* perform feature matching between two images based on their keypoints and descriptors using the **Brute force** approach which is also called **one to all**, but they differ in the way they calculate the distances between the descriptors.


### Differences:

- *find_matches* takes two images and their corresponding key points and descriptors, while *find_matches_1* takes each image's key points and descriptors separately.
- In *find_matches*, the distances between each pair of descriptors are computed once and stored in a matrix, while in *find_matches_1*, the distances between each descriptor in the first image and all descriptors in the second image are computed for each descriptor in the first image.
- *find_matches* uses **np.argmin** to find the index of the closest descriptor in the second image for each descriptor in the first image, while *find_matches_1* uses **np.linalg.norm** to compute the distances between each descriptor in the first image and all descriptors in the second image, and then uses **np.argmin** to find the index of the closest descriptor in the second image for each descriptor in the first image.

### Advantages of find_matches (the first function):

- It only computes the distances between descriptors once, which can be faster if the number of descriptors is large.
- It takes two images and their corresponding key points and descriptors, which can be more convenient if the images are already loaded and the key points and descriptors are computed elsewhere.

### Disadvantages of find_matches:

- It may not be as accurate as find_matches_1 since it only looks for the closest descriptor in the second image for each descriptor in the first image, while there may be a better match that is further away.
- It requires more memory since it stores the distances between descriptors in a matrix.

### Advantages of find_matches_1 (The second function):

- It can be more accurate since it considers all descriptors in the second image for each descriptor in the first image.
- It requires less memory since it only computes the distances between each descriptor in the first image and all descriptors in the second image, rather than storing all distances in a matrix.

### Disadvantages of find_matches_1:

- It can be slower if the number of descriptors is large since it computes the distances between each descriptor in the first image and all descriptors in the second image for each descriptor in the first image.
- It takes the key points and descriptors for each image separately, which may require extra steps if they are not computed yet.




In [ ]:
def run_all_find_matchers(Image_1, Image_2, keypoints_SIFT_1, keypoints_SIFT_2, descriptors_SIFT_1, descriptors_SIFT_2, threshold_1, threshold_2, x=30):
    """
    Runs three different matching functions and plots the results.

    Args:
        Image_1: first image
        Image_2: second image
        keypoints_SIFT_1: keypoints of the first image
        keypoints_SIFT_2: keypoints of the second image
        descriptors_SIFT_1: descriptors of the first image
        descriptors_SIFT_2: descriptors of the second image
        x: optional parameter for font size in the plot titles (default: 30)

    Returns:
        good_matches_1: list of good matches found using the first function
        matched_image_1: image showing the matches found using the first function
        good_matches_2: list of good matches found using the second function
        matched_image_2: image showing the matches found using the second function
        good_matches_3: list of good matches found using the third function (using built-in)
        matched_image_3: image showing the matches found using the third function (using built-in)
    """
    good_matches_1, matched_image_1 = find_matches(
        Image_1, Image_2, keypoints_SIFT_1, keypoints_SIFT_2, descriptors_SIFT_1, descriptors_SIFT_2, threshold_1)

    good_matches_2, matched_image_2 = find_matches_1(
        Image_1, keypoints_SIFT_1, descriptors_SIFT_1, Image_2, keypoints_SIFT_2, descriptors_SIFT_2, threshold_2)

    

    print("The number of good matches found using the first function are: ", len(
        good_matches_1))
    print("The number of good matches found using the second function are: ", len(
        good_matches_2))

    fig, axs = plt.subplots(1, 2, figsize=(
        Image_2.shape[1]/25, Image_2.shape[0]/25))

    # Plot the first image
    axs[0].imshow(cv2.cvtColor(matched_image_1, cv2.COLOR_BGR2RGB))
    axs[0].axis('off')
    axs[0].set_title('First Function - Distance computed once', fontsize=x)

    # Plot the second image
    axs[1].imshow(cv2.cvtColor(matched_image_2, cv2.COLOR_BGR2RGB))
    axs[1].axis('off')
    axs[1].set_title(
        "Second Function - Computed for every descriptor", fontsize=x)

    plt.subplots_adjust(wspace=0.05, hspace=0.05)
    plt.show()
    return good_matches_1, matched_image_1, good_matches_2, matched_image_2


## Updating the match finding function to implement 1NN/2NN ratio test

By doing this, we can remove some ambiguous correspondences inside the *match finding* functions. <br>
<br>
The 1NN/2NN ratio test is a technique used in computer vision and image processing to remove ambiguous matches between the features of two images.<br><br>

When finding matches between the features of two images, a common technique is to compare the descriptors of each feature and find the closest match in the other image. However, sometimes the closest match is not the correct match and can lead to false matches. To address this, the 1NN/2NN ratio test compares the distances of the two closest matches for each feature, and removes the match if the distance ratio between the two matches is above a certain threshold.<br><br>

The 1NN/2NN ratio test works as follows: for each feature in the first image, we find the two closest matches in the second image. We then compute the ratio of the distance between the first and second matches. If this ratio is below a threshold (typically 0.8 or 0.7), we accept the match as valid. Otherwise, we reject the match as ambiguous.<br><br>

The 1NN/2NN ratio test removes ambiguous matches by ensuring that the best match is significantly better than the second best match. This is based on the assumption that the correct match should be much closer to the feature than any other incorrect match. By using a ratio test, we can avoid false matches that may result from features that have similar descriptors or from noise in the images.<br><br>

In [ ]:
def find_matches_filtered(img1, img2, kp1, kp2, des1, des2, threshold, ratio=0.75):
    """
    Finds matching features between two images based on their keypoints and descriptors.

    Args:
        img1: first image
        kp1: keypoints of the first image
        des1: descriptors of the first image
        img2: second image
        kp2: keypoints of the second image
        des2: descriptors of the second image
        threshold: threshold used to filter matches based on descriptor distance
        ratio: threshold used to remove ambiguous matches based on 1NN/2NN distance ratio

    Returns:
        matches: list of matching features
        matched_image: image showing the matches between the two input images
    """
    # Check that the keypoints and descriptors are not empty
    if not (len(kp1) and len(kp2) and len(des1) and len(des2)):
        print("Empty keypoints or descriptors")
        return [], None

    # Compute the distances between each pair of descriptors
    distances = cdist(des1, des2, 'euclidean')

    # Set the threshold dynamically based on the distribution of distances
    threshold = np.median(distances) * threshold

    # Perform 1NN/2NN ratio test to remove ambiguous matches
    matches = []
    for i in range(len(des1)):
        # Find the indices of the closest and second closest descriptors in the second image
        sorted_indices = np.argsort(distances[i])
        closest_idx, second_closest_idx = sorted_indices[0], sorted_indices[1]

        # Calculate the distance ratio between the closest and second closest descriptors
        distance_ratio = distances[i][closest_idx] / distances[i][second_closest_idx]

        # If the distance ratio is below the threshold and the closest distance is below the threshold, add the match
        closest_distance = distances[i][closest_idx]
        if distance_ratio < ratio and closest_distance < threshold:
            matches.append(cv2.DMatch(i, closest_idx, closest_distance))

    # Draw the matches as lines between corresponding key points
    if matches:
        matched_image = cv2.drawMatches(img1, kp1, img2, kp2, matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    else:
        matched_image = None

    return matches, matched_image

In [ ]:
def find_matches_1_filtered(img1, kp1, des1, img2, kp2, des2, threshold, ratio=0.75):
    """
    Finds matches between keypoints in two images using Euclidean distance and 1NN/2NN ratio test.

    Args:
        img1: first image
        kp1: keypoints in first image
        des1: descriptors for keypoints in first image
        img2: second image
        kp2: keypoints in second image
        des2: descriptors for keypoints in second image
        threshold: distance threshold for matching keypoints
        ratio: 1NN/2NN ratio threshold for removing ambiguous matches (default 0.75)

    Returns:
        matches: list of matched keypoints (cv2.DMatch objects)
        matched_image: image with matched keypoints drawn
    """
    # Check that the keypoints and descriptors are not empty
    if not (len(kp1) and len(kp2) and len(des1) and len(des2)):
        print("Empty keypoints or descriptors")
        return [], None

    # Compute the distances between each pair of descriptors
    distances = cdist(des1, des2, 'euclidean')

    # Set the threshold dynamically based on the distribution of distances
    threshold = np.median(distances) * threshold

    matches = []
    for i in range(len(kp1)):
        # Compute the distances between the descriptor i from image 1 and all descriptors from image 2
        distances_i = np.linalg.norm(des2 - des1[i], axis=1)

        # Find the indices of the two closest descriptors in the second image
        indices = np.argsort(distances_i)[:2]

        # Check if the distance ratio is below the threshold
        if distances_i[indices[0]] < ratio * distances_i[indices[1]] and distances_i[indices[0]] < threshold:
            matches.append(cv2.DMatch(i, indices[0], distances_i[indices[0]]))

    matched_image = cv2.drawMatches(
        img1, kp1, img2, kp2, matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

    return matches, matched_image


In [ ]:
def find_matches_using_builtin_function(image1, image2, keypoints1, descriptors1, keypoints2, descriptors2, threshold):
    """
    Finds good matches between the descriptors of two input images using the brute force matcher algorithm.

    Args:
        image1: first input image
        image2: second input image
        keypoints1: keypoints of the first image
        descriptors1: descriptors of the first image
        keypoints2: keypoints of the second image
        descriptors2: descriptors of the second image
        max_distance: maximum allowed distance between the descriptors

    Returns:
        good_matches: list of good matches between the descriptors of the two images
        matched_image: image showing the good matches between the two input images
    """
    # Create a brute force matcher object
    matcher = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)

    # Match the descriptors of the two images
    matches = matcher.knnMatch(descriptors1, descriptors2, k=2)

    # Filter the matches based on the distance between the descriptors
    good_matches = []
    for m, n in matches:
        if m.distance < threshold * n.distance:
            good_matches.append([m])

    # Draw the matches on the images
    matched_image = cv2.drawMatchesKnn(image1, keypoints1, image2, keypoints2,
                                       good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

    return good_matches, matched_image


In [ ]:
def run_all_find_matchers_filtered(Image_1, Image_2, keypoints_SIFT_1, keypoints_SIFT_2, descriptors_SIFT_1, descriptors_SIFT_2, threshold_1, threshold_2, threshold_3, x=30):
    """
    Runs three different filtered matching functions and plots the results.

    Args:
        Image_1: first image
        Image_2: second image
        keypoints_SIFT_1: keypoints of the first image
        keypoints_SIFT_2: keypoints of the second image
        descriptors_SIFT_1: descriptors of the first image
        descriptors_SIFT_2: descriptors of the second image
        x: optional parameter for font size in the plot titles (default: 30)

    Returns:
        good_matches_1_filtered: list of good matches found using the first function
        matched_image_1_filtered: image showing the matches found using the first function
        good_matches_2_filtered: list of good matches found using the second function
        matched_image_2_filtered: image showing the matches found using the second function
        good_matches_3_built_in: list of good matches found using the third function (using built-in)
        matched_image_3_built_in: image showing the matches found using the third function (using built-in)
    """
    good_matches_1_filtered, matched_image_1_filtered = find_matches_filtered(
        Image_1, Image_2, keypoints_SIFT_1, keypoints_SIFT_2, descriptors_SIFT_1, descriptors_SIFT_2, threshold_1)

    good_matches_2_filtered, matched_image_2_filtered = find_matches_1_filtered(
        Image_1, keypoints_SIFT_1, descriptors_SIFT_1, Image_2, keypoints_SIFT_2, descriptors_SIFT_2, threshold_2)

    good_matches_3_built_in, matched_image_3_built_in = find_matches_using_builtin_function(
        Image_1, Image_2, keypoints_SIFT_1, descriptors_SIFT_1, keypoints_SIFT_2, descriptors_SIFT_2, threshold_3)

    print("The number of good matches found after using 1NN/2NN ratio test on the first function are: ", len(
        good_matches_1_filtered))
    print("The number of good matches found after using 1NN/2NN ratio test on the second function are: ", len(
        good_matches_2_filtered))
    print("The number of good matches found using the third function (using built-in) are: ",
          len(good_matches_3_built_in))

    fig, axs = plt.subplots(1, 3, figsize=(
        Image_2.shape[1]/25, Image_2.shape[0]/25))

    # Plot the first image
    axs[0].imshow(cv2.cvtColor(matched_image_1_filtered, cv2.COLOR_BGR2RGB))
    axs[0].axis('off')
    axs[0].set_title('First Function - Distance computed once', fontsize=x)

    # Plot the second image
    axs[1].imshow(cv2.cvtColor(matched_image_2_filtered, cv2.COLOR_BGR2RGB))
    axs[1].axis('off')
    axs[1].set_title(
        "Second Function -  Computed for every descriptor", fontsize=x)

    # Plot the third image
    axs[2].imshow(cv2.cvtColor(matched_image_3_built_in, cv2.COLOR_BGR2RGB))
    axs[2].axis('off')
    axs[2].set_title("Using Built-in Functions", fontsize=x)
    plt.subplots_adjust(wspace=0.05, hspace=0.05)
    plt.show()

    return good_matches_1_filtered, matched_image_1_filtered, good_matches_2_filtered,\
          matched_image_2_filtered, good_matches_3_built_in, matched_image_3_built_in


## Display the keypoints on the image

In [ ]:
def drawKeypoints(image, keypoints):
    """
    Draws keypoints on an image.

    Args:
        image: image to draw keypoints on
        keypoints: keypoints to draw

    Returns:
        result: image with keypoints drawn on it
    """
    result = cv2.drawKeypoints(image, keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    return result


In [ ]:
def plot_keypoints_subplot(image, result1, result2, result3, result4, x=20):
    """
    Plots the results.

    Args:
        image: image to draw keypoints on
        result1: result of draw_keypoints function
        result2: result of draw_keypoints function
        result3: result of draw_keypoints function
        result4: result of draw_keypoints function
        x: optional parameter for font size in the plot titles (default: 20) 

    Returns:
        Nothing
    """
    fig, ax = plt.subplots(2, 2, figsize=(
        image.shape[1]/30, image.shape[0]/30))
    ax[0, 0].imshow(cv2.cvtColor(result1, cv2.COLOR_BGR2RGB), aspect='equal')
    ax[0, 0].axis('off')
    ax[0, 1].imshow(cv2.cvtColor(result2, cv2.COLOR_BGR2RGB), aspect='equal')
    ax[0, 1].axis('off')
    ax[1, 0].imshow(cv2.cvtColor(result3, cv2.COLOR_BGR2RGB), aspect='equal')
    ax[1, 0].axis('off')
    ax[1, 1].imshow(cv2.cvtColor(result4, cv2.COLOR_BGR2RGB), aspect='equal')
    ax[1, 1].axis('off')
    ax[0, 0].set_title('All Key points', fontsize=x)
    ax[0, 1].set_title('Best 1000 keypoints', fontsize=x)
    ax[1, 0].set_title('SIFT parameters', fontsize=x)
    ax[1, 1].set_title('BRISK', fontsize=x)
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.show()


## Driver Code

## Reference Image **Van Gogh**

In [ ]:
# Import the Image
ref_image = cv2.imread('data/img_ref.jpg')

keypoints, descriptors,\
    keypoints_np, N_keypoints, N_features,\
    keypoints_SIFT, descriptors_SIFT,\
    keypoints_BRISK, descriptors_BRISK = run_all_extractors(ref_image)

# Generate the image data for each image

result1 = drawKeypoints(ref_image, keypoints)
result2 = drawKeypoints(ref_image, N_keypoints)
result3 = drawKeypoints(ref_image, keypoints_SIFT)
result4 = drawKeypoints(ref_image, keypoints_BRISK)

plot_keypoints_subplot(ref_image, result1, result2, result3, result4)


## **ESIREM logo**

In [ ]:
# Import the Image and convert it to Gray-Scale
E_logo = cv2.imread('data/esirem.jpg')

keypoints_E, descriptors_E,\
    keypoints_np_E, N_keypoints_E, N_features_E,\
    keypoints_SIFT_E, descriptors_SIFT_E,\
    keypoints_BRISK_E, descriptors_BRISK_E = run_all_extractors(E_logo)

# Generate the image data for each image

result1_E = drawKeypoints(E_logo, keypoints_E)
result2_E = drawKeypoints(E_logo, N_keypoints_E)
result3_E = drawKeypoints(E_logo, keypoints_SIFT_E)
result4_E = drawKeypoints(E_logo, keypoints_BRISK_E)

plot_keypoints_subplot(E_logo, result1_E, result2_E, result3_E, result4_E)


## Img_1

In [ ]:
# Import the Image and convert it to Gray-Scale
img_1 = cv2.imread('data/img_1.jpg')

keypoints_1, descriptors_1,\
    keypoints_np_1, N_keypoints_1, N_features_1,\
    keypoints_SIFT_1, descriptors_SIFT_1,\
    keypoints_BRISK_1, descriptors_BRISK_1 = run_all_extractors(img_1)

# Generate the image data for each image

result1_1 = drawKeypoints(img_1, keypoints_1)
result2_1 = drawKeypoints(img_1, N_keypoints_1)
result3_1 = drawKeypoints(img_1, keypoints_SIFT_1)
result4_1 = drawKeypoints(img_1, keypoints_BRISK_1)

plot_keypoints_subplot(img_1, result1_1, result2_1, result3_1, result4_1, x=75)


## img_2

In [ ]:
# Import the Image
img_2 = cv2.imread('data/img_2.png')

keypoints_2, descriptors_2,\
    keypoints_np_2, N_keypoints_2, N_features_3,\
    keypoints_SIFT_2, descriptors_SIFT_2,\
    keypoints_BRISK_2, descriptors_BRISK_2 = run_all_extractors(img_2)

# Generate the image data for each image

result1_2 = drawKeypoints(img_2, keypoints_2)
result2_2 = drawKeypoints(img_2, N_keypoints_2)
result3_2 = drawKeypoints(img_2, keypoints_SIFT_2)
result4_2 = drawKeypoints(img_2, keypoints_BRISK_2)

plot_keypoints_subplot(img_2, result1_2, result2_2, result3_2, result4_2, x=50)


## img_3

In [ ]:
# Import the Image
img_3 = cv2.imread('data/img_3.jpg')

keypoints_3, descriptors_3,\
    keypoints_np_3, N_keypoints_3, N_features_3,\
    keypoints_SIFT_3, descriptors_SIFT_3,\
    keypoints_BRISK_3, descriptors_BRISK_3 = run_all_extractors(img_3)

# Generate the image data for each image

result1_3 = drawKeypoints(img_3, keypoints_3)
result2_3 = drawKeypoints(img_3, N_keypoints_3)
result3_3 = drawKeypoints(img_3, keypoints_SIFT_3)
result4_3 = drawKeypoints(img_3, keypoints_BRISK_3)

plot_keypoints_subplot(img_3, result1_3, result2_3, result3_3, result4_3, x=50)


## Img_4

In [ ]:
# Import the Image
img_4 = cv2.imread('data/img_4.jpg')

keypoints_4, descriptors_4,\
    keypoints_np_4, N_keypoints_4, N_features_4,\
    keypoints_SIFT_4, descriptors_SIFT_4,\
    keypoints_BRISK_4, descriptors_BRISK_4 = run_all_extractors(img_4)

# Generate the image data for each image

result1_4 = drawKeypoints(img_4, keypoints_4)
result2_4 = drawKeypoints(img_4, N_keypoints_4)
result3_4 = drawKeypoints(img_4, keypoints_SIFT_4)
result4_4 = drawKeypoints(img_4, keypoints_BRISK_4)

plot_keypoints_subplot(img_4, result1_4, result2_4, result3_4, result4_4, x=75)


1. **Question:** Are there differences regarding the number of detected keypoints using the default parameters?. <br><br>

**Answer:** Yes, there will be differences in the number of detected keypoints in the two codes.<br>

- extract_features_SIFT_parameters(image) is using SIFT with nfeatures=1000, which means that it will detect up to 1000 keypoints in the image, regardless of their quality.<br> 

- On the other hand, extract_features(img, n=1000) is using SIFT without any limit on the number of keypoints detected. It then sorts the detected keypoints by their response and only selects the top N keypoints, where N is a parameter passed to the function.<br>

 - This means that the number of keypoints selected in the extract_features will depend on the quality of the detected keypoints, and may be less than or greater than 1000 depending on the image and the value of n.<br>

 - On the other hand we have BRISK, whose results are not as good as SIFT but is an alternative available. Ofcourse, now that SIFT is free to use, we should use SIFT

## Unfiltered -- Without 1NN/2NN Ratio test

## Matches for image one and reference image

In [ ]:
img_1_good_matches_1, img_1_matched_image_1, \
    img_1_good_matches_2, img_1_matched_image_2, = \
    run_all_find_matchers(ref_image, img_1, keypoints_SIFT,
                          keypoints_SIFT_1, descriptors_SIFT, 
                          descriptors_SIFT_1, threshold_1 = 0.5, 
                          threshold_2 = 0.5, x = 75)


## Matches for image two and reference image

In [ ]:
img_2_good_matches_1, img_2_matched_image_1, \
    img_2_good_matches_2, img_2_matched_image_2, = \
    run_all_find_matchers(ref_image, img_2, keypoints_SIFT,
                          keypoints_SIFT_2, descriptors_SIFT, 
                          descriptors_SIFT_2, threshold_1 = 0.5, 
                          threshold_2 = 0.5, x=40)


## Matches for image three and reference image

In [ ]:
img_3_good_matches_1, img_3_matched_image_1, \
    img_3_good_matches_2, img_3_matched_image_2,  = \
    run_all_find_matchers(ref_image, img_3, keypoints_SIFT,
                          keypoints_SIFT_3, descriptors_SIFT, 
                          descriptors_SIFT_3, threshold_1 = 0.5,
                          threshold_2 = 0.5)


## Matches for image four and reference image

In [ ]:
img_4_good_matches_1, img_4_matched_image_1, \
    img_4_good_matches_2, img_4_matched_image_2,  = \
    run_all_find_matchers(ref_image, img_4, keypoints_SIFT,
                          keypoints_SIFT_4, descriptors_SIFT, 
                          descriptors_SIFT_4, threshold_1 = 0.5, 
                          threshold_2 = 0.5, x=60)


## Filtered -- With 1NN/2NN Ratio test

## Filtered Matches for image one and reference image

In [ ]:
img_1_good_matches_1_filtered, img_1_matched_image_1_filtered, \
    img_1_good_matches_2_filtered, img_1_matched_image_2_filtered, \
    img_1_good_matches_3_built_in, img_1_matched_image_3_built_in = \
    run_all_find_matchers_filtered(ref_image, img_1, keypoints_SIFT,
                          keypoints_SIFT_1, descriptors_SIFT, 
                          descriptors_SIFT_1, threshold_1 = 0.5, 
                          threshold_2 = 0.5, threshold_3 = 0.75, x = 55)

## Filtered Matches for image two and reference image

In [ ]:
img_2_good_matches_1_filtered, img_2_matched_image_1_filtered, \
    img_2_good_matches_2_filtered, img_2_matched_image_2_filtered, \
    img_2_good_matches_3_built_in, img_2_matched_image_3_built_in = \
    run_all_find_matchers_filtered(ref_image, img_2, keypoints_SIFT,
                          keypoints_SIFT_2, descriptors_SIFT, 
                          descriptors_SIFT_2, threshold_1 = 0.5, 
                          threshold_2 = 0.5, threshold_3 = 0.75, x=30)


## Filtered Matches for image three and reference image

In [ ]:
img_3_good_matches_1_filtered, img_3_matched_image_1_filtered, \
    img_3_good_matches_2_filtered, img_3_matched_image_2_filtered, \
    img_3_good_matches_3_built_in, img_3_matched_image_3_built_in = \
    run_all_find_matchers_filtered(ref_image, img_3, keypoints_SIFT,
                          keypoints_SIFT_3, descriptors_SIFT, 
                          descriptors_SIFT_3, threshold_1 = 0.5,
                          threshold_2 = 0.5, threshold_3 = 0.75, x = 25)

## Filtered Matches for image four and reference image

In [ ]:
img_4_good_matches_1_filtered, img_4_matched_image_1_filtered, \
    img_4_good_matches_2_filtered, img_4_matched_image_2_filtered, \
    img_4_good_matches_3_built_in, img_4_matched_image_3_built_in = \
    run_all_find_matchers_filtered(ref_image, img_4, keypoints_SIFT,
                          keypoints_SIFT_4, descriptors_SIFT, 
                          descriptors_SIFT_4, threshold_1 = 0.5, 
                          threshold_2 = 0.5, threshold_3 = 0.75, x=50)


### The difference between filter and unfiltered matches

In [ ]:
print("The difference between the Filtered and Unfiltered Matches Response for the 1st picture is: \n",\
      len(img_1_good_matches_1)  - len(img_1_good_matches_1_filtered), 'ambiguous matches')

print("The difference between the Filtered and Unfiltered Matches Response for the second picture is: \n",\
      len(img_2_good_matches_1)  - len(img_2_good_matches_1_filtered), 'ambiguous matches')

print("The difference between the Filtered and Unfiltered Matches Response for the 3rd picture is: \n",\
      len(img_3_good_matches_1)  - len(img_3_good_matches_1_filtered), 'ambiguous matches')

print("The difference between the Filtered and Unfiltered Matches Response for the 4th picture is: \n",\
      len(img_4_good_matches_1)  - len(img_4_good_matches_1_filtered), 'ambiguous matches')

1. We can see that by using the 1NN/2NN ratio test, we are able to remove many ambiguous matches
2. Also, we can see that after applying the $\frac{1NN}{2NN}$ ratio test, the results are almost the same as the built in functions provided by OpenCV library

# Part II - Robust 2D Homography Model Fitting

## Sources:
[**Prof. Dr. Cyrill Stachniss**](https://www.ipb.uni-bonn.de/people/cyrill-stachniss/)<br><br>

[**Photogrammetry** by Prof. Dr. Cyrill Stachniss](https://www.ipb.uni-bonn.de/photo12-2021/)<br><br>

[**Direct Linear Transform** -DLT- for Camera Calibration and Localization Slides by Prof. Dr. Cyrill Stachniss](https://www.ipb.uni-bonn.de/html/teaching/photo12-2021/2021-pho1-21-DLT.pptx.pdf)  <br><br>

[**RANSAC** – Random Sample Consensus by Prof. Dr. Cyrill Stachniss](https://www.ipb.uni-bonn.de/html/teaching/photo12-2021/2021-pho2-06-ransac.pptx.pdf)<br><br>

[**Orthophotos** by Prof. Dr. Cyrill Stachniss](https://www.ipb.uni-bonn.de/html/teaching/photo12-2021/2021-pho2-11-orthophoto.pptx.pdf)<br><br>

[**Senior Lecturer Dr NAYEEM**](https://www.cs.umd.edu/~nayeem/)<br><br>

[**Homography** by Mohammad Nayeem Teli](https://www.cs.umd.edu/class/fall2019/cmsc426-0201/files/16_Homography-estimation-and-decomposition.pdf)


## Homography estimation using DLT

We will now estimate a transformation between each two images based on the found matches in the previous step. Since we are observing a planar object this transformation can be represented by an homography. <br><br>

We need at least ***4*** corresponding 2D pints for fitting a 2D Homography transformation model.

<br><br>

We want to implement the normalized DLT for 2D Homography estimation. <br><br>

The normalized Direct Linear Transformation (DLT) algorithm is an extension of the standard DLT algorithm that accounts for differences in scaling and orientation between the two images. Here are the steps for the normalized DLT algorithm for 2D homography:<br>

1. Normalize the coordinates: For each set of corresponding points, compute the centroid of the points in each set and scale them so that the average distance to the centroid is sqrt(2). This step helps to reduce numerical errors that can arise from large coordinate values.<br>
2. Construct the matrix A: For each corresponding pair of normalized points *(x_i', y_i')* and *(x_i, y_i)*, construct a row in the matrix A as follows:<br>
[ 0 0 0 -x_i' -y_i' -1 y_ix_i' y_iy_i' y_i ] <br>
A_i = [ x_i' y_i' 1 0 0 0 -x_ix_i' -x_iy_i' -x_i ]<br>

3. Compute the homography matrix H: Compute the null space of A using Singular Value Decomposition (SVD) and obtain the vector h that corresponds to the column of V that corresponds to the smallest singular value. Reshape h into a 3x3 matrix and normalize it so that the bottom right element is 1.<br>

4. Denormalize the homography matrix: Undo the normalization that was applied in step 1 by computing the transformation that maps the normalized points to their original positions. This transformation can be computed using the centroids and scaling factors that were computed in step 1. <br>

Note: The above steps assume that there are at least four corresponding points, which is the minimum number of points needed to estimate a homography matrix. If there are more than four points, the algorithm can be extended to use more points to improve the accuracy of the estimated homography.


To apply the **normalized DLT algorithm for 2D homography using SIFT correspondences**, the steps are: <br>

1. Extract the SIFT keypoints and descriptors for both images, which we have done in the previous step.<br>

2. Use a matching algorithm, such as **brute force** matching, to find the correspondences between the SIFT descriptors in the two images. This will give you a set of matching keypoints in both images which we have also obtained in the previous step.<br>

3. Choose at least 4 matching keypoints to use in the homography estimation. You can randomly select four keypoints, or choose them based on some criteria such as the number of inliers or the distribution of matches across the image.<br>

4. Normalize the coordinates of the chosen keypoints by subtracting the centroid and dividing by the average distance to the centroid.<br>

5. Construct the matrix A using the normalized coordinates of the chosen keypoints.<br>

6. Use SVD to solve for the null space of A, which will give you the homography matrix H.<br>

7. Denormalize the homography matrix H by reversing the normalization that was done in step 4.<br>

8. Use the homography matrix H to warp one image onto the other using image warping techniques such as perspective transformation or affine transformation.<br><br>

**Note:**  If the number of inliers is low or the matches are noisy, the homography estimation may be inaccurate. In this case, we can use RANSAC to improve the accuracy of the estimated homography.

### Using CPU

In [ ]:
def normalize_cpu(pts):

    """
    Normalize points so that the centroid is at the origin
    and the average distance from the origin is sqrt(2).
    """
    mean = np.mean(pts, axis=0)
    dist = np.sqrt(np.sum((pts - mean)**2, axis=1)).mean()
    T = np.array([[np.sqrt(2)/dist, 0, -mean[0]*np.sqrt(2)/dist],
                [0, np.sqrt(2)/dist, -mean[1]*np.sqrt(2)/dist],
                [0, 0, 1]])
    return np.hstack((pts, np.ones((pts.shape[0], 1)))) @ T.T, T

In [ ]:
def denormalize_H_cpu(H, T1, T2):
    
    """
    Denormalize the homography matrix H given the normalization
    transformations T1 and T2.

    Parameters:
        :H: Homography Matrix
        :T1 and T2: Normalization transformations
        
    Returns: 
        Denomarlized H matrix (3x3)
    """
    return np.linalg.inv(T2) @ H @ T1

In [ ]:
def normalized_dlt_cpu(keypoints1, keypoints2, matches, k=4):

    """
    Estimate the homography matrix using normalized DLT (CPU).

    :param keypoints1: representing the source points.
    :param keypoints2: a numpy array of shape (N, 2) representing the destination points.
    
    :return: Homography matrix(3x3)
    """

    # Choose at least 4 matching keypoints
    kp1 = [keypoints1[m.queryIdx].pt for m in matches]
    kp2 = [keypoints2[m.trainIdx].pt for m in matches]
    indices = np.random.choice(len(kp1), k, replace=False)
    pts1 = np.array([kp1[i] for i in indices])
    pts2 = np.array([kp2[i] for i in indices])


    # Normalize the coordinates of the chosen keypoints
    pts1_norm, T1 = normalize_cpu(pts1)
    pts2_norm, T2 = normalize_cpu(pts2)

    # Construct the matrix A
    A = []
    for i in range(pts1_norm.shape[0]):
        x1, y1, _ = pts1_norm[i]
        x2, y2, _ = pts2_norm[i]
        A.append([0, 0, 0, -x1, -y1, -1, y2*x1, y2*y1, y2])
        A.append([x1, y1, 1, 0, 0, 0, -x2*x1, -x2*y1, -x2])
    A = np.array(A)

    # Solve for the null space of A using SVD
    _, _, V = svd(A)
    H_norm = V[-1, :].reshape(3, 3)

    # Denormalize the homography matrix
    H = denormalize_H_cpu(H_norm, T1, T2)
    np.set_printoptions(suppress=True)

    return np.round(H, decimals=3)


Note that this implementation uses random sampling to select the 4 keypoints used in homography estimation, which can lead to different results each time it runs.

### Using GPU

In [ ]:
def normalize_gpu(pts):
    """
    Normalize points so that the centroid is at the origin
    and the average distance from the origin is sqrt(2).
    """
    pts = pts.float()
    mean = pts.mean(dim=0)
    dist = torch.sqrt(torch.sum((pts - mean)**2, dim=1)).mean()
    T = torch.tensor([[np.sqrt(2)/dist, 0, -mean[0]*np.sqrt(2)/dist],
                    [0, np.sqrt(2)/dist, -mean[1]*np.sqrt(2)/dist],
                    [0, 0, 1]])
    T = T.to(pts.device)
    return torch.cat((pts, torch.ones((pts.shape[0], 1), device=pts.device)), dim=1) @ T.T, T

In [ ]:
def denormalize_H_gpu(H, T1, T2):
    
    """
    Denormalize the homography matrix H given the normalization
    transformations T1 and T2.

    Parameters:
        :H: Homography Matrix
        :T1 and T2: Normalization transformations
        
    Returns: 
        Denomarlized H matrix (3x3)
    """
    return torch.inverse(T2) @ H @ T1

In [ ]:
def normalized_dlt_gpu(keypoints1, keypoints2, matches, k=4):

    """
    Estimate the homography matrix using normalized DLT (GPU).

    Parameters: 

        :param keypoints1: representing the source points.
        :param keypoints2: a numpy array of shape (N, 2) representing the destination points.
    
    Returns: 
        Homography matrix(3x3) data type = tensor
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Choose 4 matching keypoints
    matches_subset = np.random.choice(matches, k, replace=False)
    kp1 = [keypoints1[m.queryIdx].pt for m in matches_subset]
    kp2 = [keypoints2[m.trainIdx].pt for m in matches_subset]
    pts1 = torch.tensor(kp1).to(device)
    pts2 = torch.tensor(kp2).to(device)


    # Normalize the coordinates of the chosen keypoints
    pts1_norm, T1 = normalize_gpu(pts1)
    pts2_norm, T2 = normalize_gpu(pts2)

    # Construct the matrix A
    A = torch.zeros((pts1_norm.shape[0]*2, 9), device=device)
    for i in range(pts1_norm.shape[0]):
        x1, y1, _ = pts1_norm[i]
        x2, y2, _ = pts2_norm[i]
        A[i*2] = torch.tensor([0, 0, 0, -x1, -y1, -1,
                              y2*x1, y2*y1, y2], device=device)
        A[i*2+1] = torch.tensor([x1, y1, 1, 0, 0,
                                0, -x2*x1, -x2*y1, -x2], device=device)

        # Solve for the null space of A using SVD
    _, _, V = torch.svd(A)
    H_norm = V[:, -1].reshape(3, 3)

    # Denormalize the homography matrix
    H = denormalize_H_gpu(H_norm, T1, T2)

    return H

H_GPU = normalized_dlt_gpu(keypoints_SIFT, keypoints_SIFT_1, img_1_good_matches_1_filtered)
print(H_GPU)
print(np.shape(H_GPU))

H_GPU = to_numpy_array(H_GPU)

## Homography estimation using RANSAC

We will now estimate a transformation between each two images based on the found matches in the previous step. Since we are observing a planar object this transformation can be represented by an homography. <br><br>

We need at least ***4*** corresponding 2D pints for fitting a 2D Homography transformation model.

<br><br>

We want to implement RANSAC for 2D Homography estimation. <br><br>

Fischler and Bolles introduced the RANSAC algorithm in their 1981 paper, "Random Sample Consensus: A Paradigm for Model Fitting with Applications to Image Analysis and Automated Cartography." RANSAC is a general algorithm for fitting models to observed data that may contain outliers. The algorithm works by repeatedly selecting a random subset of the data and fitting a model to that subset, and then using that model to determine which of the remaining data points are inliers (consistent with the model) and which are outliers (inconsistent with the model). The process is repeated a fixed number of times, and the model with the most inliers is chosen as the best model.<br>

The steps for RANSAC algorithm are:<br>

1. Set the number of iterations, and threshold.<br>

2. For i = 1 to N, Choose a random subset of the observed data.<br>

3. Fit a model to the chosen subset.

4. Determine the inliers: for each data point not in the chosen subset, determine if the point is consistent with the model. If the point is consistent (i.e., the error is below a threshold), add it to the inliers set. <br>

5. If the number of inliers is above a threshold, re-fit the model to all the inliers and return it as the best model.<br>

6. If the number of inliers is greater than some fraction p of the total number of data points, exit the loop and return the best estimate.<br>

7. Choose the best model found while iterating N times.<br><br>

We want to modify this algorithm to estimate the Homography matrix.<br>

Here are the steps to implement RANSAC algorithm for estimating the homography matrix::<br>

1. Randomly select a minimal set of points (usually 4) from the set of observed data points.<br>

2. Compute the homography matrix using the selected points.<br>

3. For each remaining data point, calculate the distance between the point and its projection onto the estimated homography.<br>

4. Count the number of inliers (data points whose distances are less than a threshold value) and update the best homography if this number is larger than the previous iterations.<br>

5. Repeat steps 1-4 for a fixed number of iterations or until the desired number of inliers is found. <br>

6. Re-estimate the homography using all the inliers found in step 4. <br>


### Using CPU

In [ ]:
def normalized_dlt_cpu_ransac(keypoints1, keypoints2, matches, k=4, ransac_iters=100000, ransac_threshold=5.0):
    best_H = None
    best_inliers = None
    num_inliers = 0
    previous_num_inliers = -1

    while num_inliers != previous_num_inliers:
        previous_num_inliers = num_inliers

        for i in range(ransac_iters):
            # Estimate homography matrix using normalized DLT algorithm
            H = normalized_dlt_cpu(keypoints1, keypoints2, matches, k)

            # Calculate the number of inliers and outliers
            inliers = []
            outliers = []
            for m in matches:
                x1, y1 = keypoints1[m.queryIdx].pt
                x2, y2 = keypoints2[m.trainIdx].pt
                pt1_norm_h = np.array([x1, y1, 1.0])
                pt2_norm_h_est = np.dot(H, pt1_norm_h)
                pt2_norm_est = pt2_norm_h_est[:2] / pt2_norm_h_est[2]
                pt2_norm_h = np.array([x2, y2])
                dist = np.linalg.norm(pt2_norm_h - pt2_norm_est)
                if dist <= ransac_threshold:
                    inliers.append(m)
                else:
                    outliers.append(m)

            # Update the best model if the current one has more inliers
            if len(inliers) > num_inliers:
                best_H = H
                best_inliers = inliers
                num_inliers = len(inliers)

    return best_H, best_inliers

In [ ]:
def normalized_dlt_ransac_gpu(keypoints1, keypoints2, matches, num_iterations=1000, threshold=5):
    """
    Estimate the homography matrix using RANSAC algorithm.

    :param keypoints1: representing the source points.
    :param keypoints2: a numpy array of shape (N, 2) representing the destination points.
    :param num_iterations: the maximum number of iterations for RANSAC algorithm.
    :param threshold: the maximum distance from a point to the estimated homography to be considered an inlier.
    :param sample_size: the size of the random subset of points to be used for estimating the homography.
    :return: a tuple of (best_model, best_inliers), where best_model is the estimated homography and best_inliers
    is a numpy array of shape (M, 2) representing the inliers.
    """
    
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    best_H = None
    best_num_inliers = 0
    previous_num_inliers = -1

    while best_num_inliers != previous_num_inliers:
        previous_num_inliers = best_num_inliers

        for i in range(num_iterations):

            H = normalized_dlt_gpu(keypoints1, keypoints2, matches)

            # Compute the number of inliers
            num_inliers = 0
            for match in matches:
                x1, y1 = keypoints1[match.queryIdx].pt
                x2, y2 = keypoints2[match.trainIdx].pt
                pt1 = torch.tensor([x1, y1, 1], device=device)
                pt2 = torch.tensor([x2, y2, 1], device=device)
                pt2_est = H @ pt1
                pt2_est_copy = pt2_est.clone()
                pt2_est_copy /= pt2_est[2]
                error = torch.sqrt(torch.sum((pt2 - pt2_est_copy)**2)).item()
                if error < threshold:
                    num_inliers += 1

            # Update the best homography matrix if the current one is better
                if num_inliers > best_num_inliers:
                    best_H = H
                    best_num_inliers = num_inliers

            
            
    # inliner_ratio = float(np.sum(num_inliers)) / len(matches)
    # print("The Inliner Ratio is: ",inliner_ratio)
    # num_iterations_95 = np.log(1 - 0.95) / np.log(1 - inliner_ratio ** 4)
    # print("The number of iterations required for 95 percent confidence is: ", num_iterations_95)
    # num_iterations_99 = np.log(1 - 0.99) / np.log(1 - inliner_ratio ** 4)
    # print("The number of iterations required for 99 percent confidence is: ",num_iterations_99)
    
    torch.set_printoptions(precision=3, sci_mode=False)

    return best_H

In [ ]:
def ransac_homography_using_Built_in(keypoints1, keypoints2, matches, num_iterations=1000, threshold=5.0, num_inliers=4):
    """
    Computes the homography matrix using RANSAC algorithm from a set of keypoint matches.

    Args:
        keypoints1: list of cv2.KeyPoint objects in image1
        keypoints2: list of cv2.KeyPoint objects in image2
        matches: list of cv2.DMatch objects
        num_iterations: maximum number of iterations to perform
        threshold: maximum distance between points for them to be considered inliers
        num_inliers: minimum number of inliers required to re-estimate the homography

    Returns:
        The homography matrix and the indices of inlier matches.
    """

    num_matches = len(matches)
    best_H = None
    best_inliers = []
    for i in range(num_iterations):
        # Randomly select 4 matches
        sample = random.sample(matches, 4)

        # Extract corresponding points
        pts1 = np.float32([keypoints1[m.queryIdx].pt for m in sample])
        pts2 = np.float32([keypoints2[m.trainIdx].pt for m in sample])

        # Compute homography matrix
        H, _ = cv2.findHomography(pts1, pts2, cv2.RANSAC, threshold)

        # Find inliers
        sample_inliers = []
        for j, match in enumerate(matches):
            pt1 = np.float32(keypoints1[match.queryIdx].pt).reshape(-1, 1)
            pt2 = np.float32(keypoints2[match.trainIdx].pt).reshape(-1, 1)

            # Append a 1 to the end of pt2 to make it a 3x1 vector
            pt2_hom = np.vstack((pt2, 1))

            # Compute distance between transformed point and its match in the other image
            pt1_hom = np.vstack((pt1, 1))
            d = np.linalg.norm(H @ pt1_hom - pt2_hom)

            # If the distance is below the threshold, the match is an inlier
            if d < threshold:
                sample_inliers.append(j)
        w = num_inliers / num_matches
# Compute the number of iterations required for 95% confidence
        p = 0.95
        k = 4
        num_iterations = int(np.log(1 - p) / np.log(1 - w ** k))
        # If enough inliers were found, re-estimate the homography using all inliers
        if len(sample_inliers) >= num_inliers:
            pts1 = np.float32([keypoints1[matches[j].queryIdx].pt for j in sample_inliers])
            pts2 = np.float32([keypoints2[matches[j].trainIdx].pt for j in sample_inliers])
            H, _ = cv2.findHomography(pts1, pts2, cv2.RANSAC, threshold)

            # If the new homography has more inliers than the previous one, update the best homography and inliers
            if len(sample_inliers) > len(best_inliers):
                best_H = H
                best_inliers = sample_inliers

    return best_H, best_inliers

1. **Question:** Discuss and indicate the parameters you need to select in order to compute one iteration of RANSAC to fit your descriptors matches. How many iterations would you select for finding the model with either 95% and 99% confidence?

**Answer:** To perform one iteration of RANSAC to fit a homography model to a set of descriptors matches, we need to select the following parameters:<br>

- ***n***: the minimum number of matches required to estimate the model. For a homography model, n should be 4 since a homography can be estimated from four correspondences.<br>

- ***k***: the number of random samples to be selected in each iteration. k should be chosen such that there is a high probability that at least one of the samples is free of outliers. A common value for k is 4, which corresponds to the minimum number of correspondences required to compute a homography.<br>

- ***Threshold***: the maximum distance between a correspondence and its projection onto the estimated model that is considered an inlier. The value of threshold should be chosen based on the expected noise in the data and the desired confidence level.

The number of iterations required to find a model with either 95% or 99% confidence depends on the percentage of inliers in the data, the number of matches, and the threshold value.<br>

The number of iterations required can be estimated using the following equation:<br>


$$ num_{iterations} = \frac{log(1 - confidence)}{log(1 - (inlier_{ratio})^n)} $$ 

where:
- ***Confidence*** is the desired confidence level, <br>

- ***$inlier_{ratio}$***: ratio of inliers to total matches, <br>

- ***n*** is the minimum number of matches required to estimate the model,<br>

- log is the natural logarithm.<br>

For example, if the inlier ratio is 0.5, the minimum number of matches is 4, and the desired confidence level is 95%, we can compute the number of iterations as: <br>
$$ num_iterations = \frac{log(1 - 0.95)}{log(1 - 0.5 ** 4)} = 11 $$

Similarly, if we want a 99% confidence level, we can compute the number of iterations as:<br>
$$ num_iterations = \frac{log(1 - 0.99)}{log(1 - 0.5 ** 4)} = 17 $$

## Wrapping one image to another Image

In [ ]:
def render_warped_texture(H, img_ref, img_tgt, patch_texture):
    # Warp patch texture that will be placed in the selected region with the estimated homography transformation H
    h, w, _ = img_ref.shape
    patch_texture_res = cv2.resize(patch_texture, (w, h))
    warped_patch = cv2.warpPerspective(patch_texture_res, H, (img_tgt.shape[1], img_tgt.shape[0]))

    # Remove the pixels of the foreground that will be replaced (we keep only background)
    mask = (255 * np.ones((h, w))).astype(np.uint8)
    mask_warped = cv2.warpPerspective(mask, H, (img_tgt.shape[1], img_tgt.shape[0]), flags=cv2.INTER_NEAREST)
    mask_background = (1 - mask_warped / 255).astype(np.uint8)
    mask_background = cv2.merge((mask_background, mask_background, mask_background))
    background = cv2.multiply(mask_background, img_tgt)

    # Blend images to a single frame and display
    blend_img = cv2.add(background, warped_patch)

    return blend_img

In [ ]:
def apply_wrapping(ref_image, original_image, new_image):
    
    sift = cv2.xfeatures2d.SIFT_create()
    matcher = cv2.BFMatcher()

    reference_keypoints, reference_descriptors = sift.detectAndCompute(ref_image, None)

    original_keypoints, original_descriptors = sift.detectAndCompute(original_image, None)
    matches = matcher.knnMatch(reference_descriptors, original_descriptors, k=2)

    # Apply ratio test to filter out poor matches
    good_matches = []
    for m,n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    reference_points = np.float32([reference_keypoints[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    original_points = np.float32([original_keypoints[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

    H, mask = cv2.findHomography(reference_points, original_points, cv2.RANSAC, 5.0)
    result = render_warped_texture(H, ref_image, original_image, new_image)
    return  result

### Print the final results

In [ ]:
def plot_keypoints_subplot(image, result1, result2, result3, result4, x=20):
    """
    Plots the results.

    Args
        result1: 
        result2: 
        result3: 
        result4: 
        x: optional parameter for font size in the plot titles (default: 20) 

    Returns:
        Nothing
    """
    fig, ax = plt.subplots(2, 2, figsize=(
        image.shape[1]/30, image.shape[0]/30))
    ax[0, 0].imshow(cv2.cvtColor(result1, cv2.COLOR_BGR2RGB), aspect='equal')
    ax[0, 0].axis('off')
    ax[0, 1].imshow(cv2.cvtColor(result2, cv2.COLOR_BGR2RGB), aspect='equal')
    ax[0, 1].axis('off')
    ax[1, 0].imshow(cv2.cvtColor(result3, cv2.COLOR_BGR2RGB), aspect='equal')
    ax[1, 0].axis('off')
    ax[1, 1].imshow(cv2.cvtColor(result4, cv2.COLOR_BGR2RGB), aspect='equal')
    ax[1, 1].axis('off')
    ax[0, 0].set_title('Image 1', fontsize=x)
    ax[0, 1].set_title('Image 2', fontsize=x)
    ax[1, 0].set_title('Image 3', fontsize=x)
    ax[1, 1].set_title('Image 4', fontsize=x)
    plt.subplots_adjust(wspace=0.05, hspace=0.05)
    plt.show()
  

## Driver Code

### CPU

In [ ]:
H_CPU = normalized_dlt_cpu(keypoints_SIFT, keypoints_SIFT_1, img_1_good_matches_1_filtered)
print(H_CPU)
print(np.shape(H_CPU))

pic = render_warped_texture(H_CPU, ref_image, img_1, E_logo)
plt.figure(figsize =(20 ,10))
plt.imshow( cv2.cvtColor(pic , cv2.COLOR_BGR2RGB ) )
plt.axis('off')
plt.show()

### RANSAC algorithm for estimating the homography matrix using DLT Homography estimation

In [ ]:
H_best = normalized_dlt_ransac_gpu(keypoints_SIFT, keypoints_SIFT_1, img_1_good_matches_1_filtered)

# Convert to numpy array
H_best = to_numpy_array(H_best)
print(H_best)
print(np.shape(H_best))
pic = render_warped_texture(H_best, ref_image, img_1, E_logo)

plt.figure(figsize =(20 ,10))
plt.imshow( cv2.cvtColor(pic , cv2.COLOR_BGR2RGB ) )
plt.axis('off')
plt.show()

In [ ]:
best_HH, best_inliers = normalized_dlt_cpu_ransac(keypoints_SIFT, keypoints_SIFT_4, img_4_good_matches_1, k=4, ransac_iters=1000, ransac_threshold=5.0)
print(best_HH)



pic = render_warped_texture(best_HH, ref_image, img_4, E_logo)
plt.figure(figsize =(20 ,10))
plt.imshow( cv2.cvtColor(pic , cv2.COLOR_BGR2RGB ) )
plt.axis('off')
plt.show()

### Wrapping With Homography matrix using Built in function from OpenCV

In [ ]:
best_H, best_inliers = ransac_homography_using_Built_in(keypoints_SIFT, keypoints_SIFT_3, img_3_good_matches_1_filtered, num_iterations=1000, threshold=5.0, num_inliers=4)
print(best_H)
pic = render_warped_texture(best_H, ref_image, img_3, E_logo)

plt.figure(figsize =(20 ,10))
plt.imshow( cv2.cvtColor(pic , cv2.COLOR_BGR2RGB ) )
plt.axis('off')
plt.show()

## Applying wrapping to all images
### Final Result

In [ ]:
E_logo_2 = cv2.imread('data/Ferrari front.jpg')
E_logo_3 = cv2.imread('data/Mercedes-w14.jpg')

result_1 = apply_wrapping(ref_image, img_1, E_logo)
result_2 = apply_wrapping(ref_image, img_2, E_logo)
result_3 = apply_wrapping(ref_image, img_3, E_logo)
result_4 = apply_wrapping(ref_image, img_4, E_logo)
plot_keypoints_subplot(img_1, result_1, result_2, result_3, result_4, x=50)

result_1 = apply_wrapping(ref_image, img_1, E_logo_2)
result_2 = apply_wrapping(ref_image, img_2, E_logo_2)
result_3 = apply_wrapping(ref_image, img_3, E_logo_2)
result_4 = apply_wrapping(ref_image, img_4, E_logo_2)
plot_keypoints_subplot(img_1, result_1, result_2, result_3, result_4, x=50)

result_1 = apply_wrapping(ref_image, img_1, E_logo_3)
result_2 = apply_wrapping(ref_image, img_2, E_logo_3)
result_3 = apply_wrapping(ref_image, img_3, E_logo_3)
result_4 = apply_wrapping(ref_image, img_4, E_logo_3)
plot_keypoints_subplot(img_1, result_1, result_2, result_3, result_4, x=50)

# Bonus TASK
## With FAST and ORB

In [ ]:
def FAST_ORB(ref_image, original_image, new_image):
    
    fast = cv2.FastFeatureDetector_create()
    orb = cv2.ORB_create()
    matcher = cv2.BFMatcher()

    reference_keypoints = fast.detect(ref_image, None)
    reference_keypoints, reference_descriptors = orb.compute(ref_image, reference_keypoints)

    original_keypoints = fast.detect(original_image, None)
    original_keypoints, original_descriptors = orb.compute(original_image, original_keypoints)

    matches = matcher.knnMatch(reference_descriptors, original_descriptors, k=2)

    # Apply ratio test to filter out poor matches
    good_matches = []
    for m,n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    reference_points = np.float32([reference_keypoints[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    original_points = np.float32([original_keypoints[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

    H, mask = cv2.findHomography(reference_points, original_points, cv2.RANSAC, 5.0)
    result = render_warped_texture(H, ref_image, original_image, new_image)
    return  result, reference_keypoints, reference_descriptors, original_keypoints, original_descriptors, good_matches, H

FAST_ORB, reference_keypoints, \
reference_descriptors, original_keypoints,\
original_descriptors, good_matches, H_FAST_ORB = \
        FAST_ORB(ref_image, img_2, E_logo)


plt.figure()
plt.imshow( cv2.cvtColor(FAST_ORB , cv2.COLOR_BGR2RGB ) )
plt.axis('off')
plt.title('Esirum Logo Wrapped Using FAST and ORB')
plt.show()

plt.figure()
plt.imshow( cv2.cvtColor(drawKeypoints(ref_image, reference_keypoints) , cv2.COLOR_BGR2RGB ) )
plt.axis('off')
plt.title('Key points of FAST on reference image')
plt.show()

matched_image = cv2.drawMatches(ref_image, reference_keypoints, img_2, original_keypoints, good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.figure()
plt.imshow( cv2.cvtColor(matched_image , cv2.COLOR_BGR2RGB ) )
plt.axis('off')
plt.title('Matches reference image and Image 2 using ORB detector')
plt.show()